In [ ]:
m = om.Image()
m.read_nd2_file('/Users/jz-rolling/Desktop/OMEGA/demo_data/mild_clumping.nd2')
m.config = om.config.Preset_configurations_default().config
m.enhance_brightfield()
m.enhance_fluorescence()
m.locate_clusters()
m.cluster_segmentation(method='annotation', 
                       model=None)
#m.cell_segmentation(split_branches=True,threshold=40)
#om.io.plot_segmented_image(m,savefig=True,
                           #filename='/Users/jz-rolling/Desktop/segmented.png',lw=0.3)
boundaries_metics = []
counter=0
is_True=1

for k,c in m.clusters.items():
    for i,b in c.boundary_pairwise_dict.items():
        img = boundary_annotation_pic(b,c)
        metrics = list(b.metrics.values())
        boundaries_metics.append(metrics)
        om.io.tifffile.imsave('/Users/jz-rolling/Desktop/OMEGA/Boundary_training_set/Training_set_2/sample_{}.tif'.format(counter),
                              img,
                              imagej=True)
        counter+=1

boundaries_metics = np.array(boundaries_metics)
df = pd.DataFrame(boundaries_metics, columns=list(b.metrics.keys()))
df.to_excel('/Users/jz-rolling/Desktop/OMEGA/Boundary_training_set/Training_set_2/training_set2_data.xls')

In [ ]:
boundaries_metics = []
counter=0
is_True=1

for k,c in m.clusters.items():
    for i,b in c.boundary_pairwise_dict.items():
        img = boundary_annotation_pic(b,c)
        metrics = list(b.metrics.values())
        boundaries_metics.append(metrics)
        om.io.tifffile.imsave('/Users/jz-rolling/Desktop/OMEGA/Boundary_training_set/Training_set_1/sample_{}.tif'.format(counter),
                              img,
                              imagej=True)
        counter+=1

boundaries_metics = np.array(boundaries_metics)
df = pd.DataFrame(boundaries_metics, columns=list(b.metrics.keys()))
df.to_excel('/Users/jz-rolling/Desktop/OMEGA/Boundary_training_set/Training_set_1/training_set1_data.xls')




In [ ]:
class_1_id = []
class_2_id = []
for x in sorted(glob.glob('/Users/jz-rolling/Desktop/OMEGA/Boundary_training_set/Training_set_1/Class_1/*.tif')):
    class_1_id.append(int(x.split('/')[-1].split('_')[-1][:-4]))
    
for y in sorted(glob.glob('/Users/jz-rolling/Desktop/OMEGA/Boundary_training_set/Training_set_1/Class_2/*.tif')):
    class_2_id.append(int(y.split('/')[-1].split('_')[-1][:-4]))
    
df = pd.read_excel('/Users/jz-rolling/Desktop/OMEGA/Boundary_training_set/Training_set_1/training_set1_data.xls')
labels = np.zeros(len(df1))
labels[np.array(class_1_id)]=0
labels[np.array(class_2_id)]=1
df['label'] = labels
df.to_excel('/Users/jz-rolling/Desktop/OMEGA/Boundary_training_set/Training_set_1/training_set1_data.xls',index=False)

In [ ]:
d = StandardScaler().fit_transform(concat_df[concat_df.columns[1:-1]])
#d = concat_df[concat_df.columns[1:-1]].values
model = MLPClassifier(solver='lbfgs', alpha=1e-5, tol=0.0001, max_iter=1000,
                      hidden_layer_sizes=(5, 2), random_state=42)
model.fit(d,concat_df['label'])

In [ ]:
pk.dump(model,open('/Users/jz-rolling/Desktop/OMEGA/OMEGA_core/configurations/boundary_MLP.pk','wb'))
pk.dump(normalizer,open('/Users/jz-rolling/Desktop/OMEGA/OMEGA_core/configurations/normalizer.pk','wb'))